In [2]:
cd ..

/Users/martinscience/mount/work/utils/EBFilter


## get the files of chrom 3 for testing

In [2]:
import os
HOME = os.environ['HOME'] # set HOME to run on different Macs
HOME = f"{HOME}/mount"

def get_AB_df(chrom):
    '''
    load and reformat the AB-cache file for a chromosome
    '''

    cache_file = f'{HOME}/work/static/ref/PoN/AML_cache/{chrom}.cache'
    # the merger columns
    cols = ['Chr', 'Start', 'Alt']
    # load the file and set Chr and Start to index for proper setting of multi-index
    # print(f"Loading cache {cache_file}..")
    AB_df = pd.read_csv(cache_file, sep=',', compression='gzip', dtype={'Chr': str, 'Start': int}).set_index(cols[:2])
    AB_columns = pd.MultiIndex.from_product([['A', 'C', 'T', 'G'], ['+', '-'], ['a', 'b']], names=['var', 'strand', 'param'])
    # set multi-indexed columns
    AB_df.columns = AB_columns
    # stack the var column level for merge with the snp_df
    AB_df= AB_df.stack('var')
    # reduce the column index level to 1
    AB_df.columns = AB_df.columns.droplevel(0)
    # unset the indices and transfer to columns
    AB_df = AB_df.reset_index()
    # rename columns for merge
    AB_df.columns = cols + ['a+', 'b+', 'a-', 'b-']
    return AB_df

AB_3 = get_AB_df('3')
anno_3 = pd.read_csv('eb_file3.csv', sep='\t', dtype={'Chr': str, 'Start': int})
snp_3 = anno_3.query('not (Ref == "-" or Alt == "-")')
bed = pd.read_csv(f'{HOME}/work/static/ref/bed_files/SureSelect/hg19/SS_HAEv6r2_hg19_Padded_nochr.bed', skiprows=10, sep='\t', dtype={0: str, 1: int}, header=None)
bed_3 = bed[bed[0] == 3][[0,1]]

FileNotFoundError: [Errno 2] File b'eb_file3.csv' does not exist: b'eb_file3.csv'

### merges

In [ ]:
merge_snpAB = AB_3.merge(snp_3, on=['Chr', 'Start'], how='right')
merge_ABbed = AB_3.merge(bed_3, left_on=['Chr', 'Start'], right_on=[0,1], how='outer')


In [ ]:
na_index = anno_3.query('(EB_score != EB_score) and (Ref == "A")').index